In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

spark = (
    SparkSession.builder
    .appName("BigData")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

import sys
# CORRIGIDO: O caminho base do seu projeto dentro do contêiner jupyterlab é /app
sys.path.append("/app")

from utils.dataloader import DataLoader
from utils.workload import Workload

# --- As definições de função permanecem as mesmas ---
def executar_load(spark, base_dir, schema=None):
    loader = DataLoader(spark=spark, base_dir=base_dir, schema=schema)
    df = loader.load()
    return df

def executar_join(df1=None, df2=None):
    df_joined = DataLoader.join(df1, df2, on=["id"], how="inner")
    return df_joined

# --- EXECUÇÃO DAS WORKLOADS DE CARREGAMENTO E JUNÇÃO ---

# --- Caminhos dos dados no contêiner ---
base_ocorrencias = "/app/full_data/ocorrencias"
base_pessoas     = "/app/full_data/pessoas"

# --- Execução das workloads usando Workload.run (sem instanciar) ---

df_ocorrencias = Workload.run(
    title="Carregamento dos dados de ocorrências",
    execute_fn=lambda spark, path: executar_load(spark, path),
    spark=spark,
    path=base_ocorrencias
)

df_pessoas = Workload.run(
    title="Carregamento dos dados de pessoas",
    execute_fn=lambda spark, path: executar_load(spark, path),
    spark=spark,
    path=base_pessoas
)

df_joined = Workload.run(
    title="Junção entre ocorrências e pessoas por ID",
    execute_fn=executar_join,
    df1=df_ocorrencias,
    df2=df_pessoas
)

In [ ]:
from utils.cleaner import salvar_csv

In [ ]:
Workload.run(
    title="Geração de amostra das ocorrências (1%)",
    execute_fn=salvar_csv,
    df=df_ocorrencias,
    output_path="/app/datasample/ocorrencias",
    amostra=True
)

Workload.run(
    title="Geração de amostra das pessoas (1%)",
    execute_fn=salvar_csv,
    df=df_pessoas,
    output_path="/app/datasample/pessoas",
    amostra=True
)

In [ ]:
Workload.run(
    title="Reescrita em UTF-8 das ocorrências completas",
    execute_fn=salvar_csv,
    df=df_ocorrencias,
    output_path="/app/full_data/ocorrencias",
    amostra=False,
    sep=";",
    encoding="UTF-8"
)

Workload.run(
    title="Reescrita em UTF-8 das pessoas completas",
    execute_fn=salvar_csv,
    df=df_pessoas,
    output_path="/app/full_data/pessoas",
    amostra=False,
    sep=";",
    encoding="UTF-8"
)